In [1]:
import datetime
from datetime import datetime
from datetime import timedelta
from typing import List, Union, Dict
import pickle
import pandas as pd
import numpy as np
from keras.utils import to_categorical

# Import
df = pd.read_csv("~/CNNworkspace/millicleantestdata.csv")

# New Time - Concatenate Date & Time fields into 1 proper datetime field
# # making a DateTime column of the datetime type for easier manipulation
# df["DateTime"] = df.apply(
#     lambda x: datetime.strptime(f"{x['date']} {x['time']}", "%d/%m/%Y %H:%M:%S"),
#     axis=1,
# )

# # Remove bad Date & Time columns
df = df.drop(["date", "time"], axis=1)

# # Get a group count to give us an anchor for iterative millisecond additions
df["annotation_group_step"] = df.groupby("date_time").cumcount()
df.head(5)

# Group each datetime (each have the same second) and iteratively add 1/25 of a second
# df["date_time"] = (
#     df.groupby("date_time")
#     .apply(
#         lambda x: x.date_time
#         + (timedelta(milliseconds=(1000 / x.shape[0])) * x.annotation_group_step)
#     )
#     .reset_index(drop=True)
# )

# df

Using TensorFlow backend.


,behavior,acc_x,acc_y,acc_z,tag_id,camera_date,camera_time,temp_c,battery_voltage,metadata,date_time,annotation_group_step
0,s,-0.016,0.031,1.000,Pen11_AXY#1_S1,3/20/18,20:11:31,NaN,NaN,NaN,2021-05-17 16:28:28.000000,0
1,s,-0.016,0.047,0.985,Pen11_AXY#1_S1,3/20/18,20:11:31,NaN,NaN,NaN,2021-05-17 16:28:28.066667,0
2,s,-0.016,0.031,1.000,Pen11_AXY#1_S1,3/20/18,20:11:31,NaN,NaN,NaN,2021-05-17 16:28:28.133334,0
3,s,-0.016,0.031,0.985,Pen11_AXY#1_S1,3/20/18,20:11:31,NaN,NaN,NaN,2021-05-17 16:28:28.200001,0
4,s,-0.016,0.016,0.985,Pen11_AXY#1_S1,3/20/18,20:11:31,NaN,NaN,NaN,2021-05-17 16:28:28.266668,0


# Get behavior counts

In [2]:
bcounts = df['behavior'].value_counts()
bdict = bcounts.to_dict() # outputs a dict with key = behavior, value = count
 
min_val = [keys for keys,values in bdict.items() if values == min(bdict.values())] #outputs all behaviors that have
            #the fewest rows of data. now if the list is longer than a single value, we will test both for consecutive
    #window length = the value_counts() of the min_val behavior
print(min_val)



['t']


In [8]:
def one_hot_coder(df):
    total_behaviors = list(df.behavior.unique())
    num_classes = len(total_behaviors)
    mapping = {}
    for x in range(num_classes):
        mapping[total_behaviors[x]] = x
    for x in range(num_classes):
        total_behaviors[x] = mapping[total_behaviors[x]]
    #one_hot_encode = to_categorical(mapping[])
    return mapping

def one_hot_labeler(mapper, behavior):
    one_hot_label = to_categorical(mapper[behavior])
    return one_hot_label

themap = one_hot_coder(df)
print(themap)
print(one_hot_labeler(themap,'w'))

{'s': 0, 'l': 1, 't': 2, 'c': 3, 'a': 4, 'd': 5, 'i': 6, 'w': 7}
[0. 0. 0. 0. 0. 0. 0. 1.]


In [51]:
def pull_windows(df, slide: int = 1, window_length: int = 15):
    """ Pull matrix window """
    
    # production 
    if window_length > df.shape[0]:
        raise ValueError('Window larger than data given')
    
    windows = []
    number_of_rows_minus_window = df.shape[0] - window_length + 1
    
    for i in range(0, number_of_rows_minus_window, slide):
        window = df[i: i + window_length]
            
        # check if all behaviors are the same; if not ignore window
        if len(set(window.behavior)) != 1:
            continue
        
        # check if times are uniform -- equal length; if not ignore window
        # if len(set(np.ediff1d(window.DateTime))) != 1:
        #     continue
        
        windows.append(window)
        
    return windows

    
def contruct_train_test(windows, df):
    positions = ['acc_x', 'acc_y', 'acc_z']
    Xtrain, Ytrain = [], []  # have to be the same length
    # TODO - give each classifier its own index and the length should be the numpy of classifiers
    
    onehot = {
        'n': [0, 0, 0, 1],  # Example: there there is only 4 classifiers in this example and we assigned 'n' to index 3
    }
    for window in windows:
        Xtrain.append(window[positions].to_numpy())
        Ytrain.append(onehot[window.behavior.values[0]])
        
    return np.stack(Xtrain), Ytrain
    
    
windows: List[pd.DataFrame] = pull_windows(df, slide=1, window_length=10)

Xtrain, Ytrain = contruct_train_test(windows, df)
Xtrain[0], Ytrain[0]

KeyError: 's'

# Save preprocessing 

In [104]:
with open('input.pkl', 'wb') as outfile:
    pickle.dump((Xtrain, Ytrain), outfile, pickle.HIGHEST_PROTOCOL)

# Read preprocessing

In [105]:
with open('input.pkl', 'rb') as infile:
    Xtrain, Ytrain = pickle.load(infile)

Xtrain[0], Ytrain[0]

(array([[-0.016, -0.219,  0.985],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.203,  0.969],
        [-0.016, -0.203,  0.969],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.219,  0.969],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.219,  0.985]]),
 [0, 0, 0, 1])

# create model 

In [ ]:
def create_model(Xtrain, *args, *kwargs):
    pass

model = create_model(Xtrain)

# train model

In [ ]:
model = model.fit(Xtrain, Ytrain)
model.tensorflow_fuckin_save

# test model 

In [87]:
def fix_ytest(Ytest) -> np.array:
    pass

windows = pull_windows(df, slide=1, window_length=10)
Xtest, Ytest = contruct_train_test(windows)

Ytest_predicted = model.predict(Xtest)  # may not be correct - tf.keras.model should have a predict function

# Yest_predicted == Ytest?
Ytest_predicted_fixed = fix(Ytest_predicted)  # i.e. a.index(max([0, .1, .22, .7]))

# Plot results

In [94]:
# yay!